
# Heart Disease Prediction Project

## 1. Problem Statement & Goal
**Goal:** Build a machine learning model to predict whether a person is at risk of heart disease based on clinical health data.

**Dataset:** UCI Heart Disease Dataset (Cleveland).
**Task:** Binary Classification (Disease vs. No Disease).


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay

# Set plot style
sns.set(style="whitegrid")
%matplotlib inline


## 2. Data Loading

In [ ]:

# Load the dataset
data_path = "data/heart_disease_uci.csv"
df = pd.read_csv(data_path)

print(f"Dataset Shape: {df.shape}")
df.head()


## 3. Data Cleaning

In [ ]:

# Standardize column names
df.columns = [c.strip() for c in df.columns]

# Drop irrelevant columns
drop_cols = ['id', 'dataset']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# Handle missing values
for col in df.columns:
    if df[col].isnull().any():
        if pd.api.types.is_numeric_dtype(df[col]):
            median = df[col].median()
            df[col] = df[col].fillna(median)
            print(f"Filled missing values in {col} with median = {median}")
        else:
            mode = df[col].mode()[0]
            df[col] = df[col].fillna(mode)
            print(f"Filled missing values in {col} with mode = {mode}")

print("Missing values after cleaning:")
print(df.isnull().sum().sum())


## 4. Exploratory Data Analysis (EDA)

In [ ]:

# Target Distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='num', data=df)
plt.title("Target Distribution (0=No Disease, 1-4=Disease)")
plt.show()

# Correlation Matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(10, 8))
sns.heatmap(df[numeric_cols].corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()


## 5. Preprocessing

In [ ]:

# Target Preparation (Binary Classification)
# The 'num' column contains 0 for no disease, and 1-4 for different stages of disease.
# We convert this to 0 (No Disease) vs 1 (Disease).

if 'num' in df.columns:
    y = df['num'].apply(lambda x: 1 if x > 0 else 0)
    X = df.drop(columns=['num'])
else:
    # Fallback if column name is different
    y = df.iloc[:,-1].apply(lambda x: 1 if x > 0 else 0)
    X = df.iloc[:,:-1]

# One-Hot Encoding for categorical variables
X = pd.get_dummies(X, drop_first=True)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training Shape: {X_train_scaled.shape}")
print(f"Testing Shape: {X_test_scaled.shape}")


## 6. Model Training

In [ ]:

# 1. Logistic Regression
lr_params = {'C': [0.01, 0.1, 1, 10, 100]}
lr_grid = GridSearchCV(LogisticRegression(max_iter=1000), lr_params, cv=5, scoring='accuracy')
lr_grid.fit(X_train_scaled, y_train)
best_lr = lr_grid.best_estimator_
print(f"Best Logistic Regression Params: {lr_grid.best_params_}")

# 2. Decision Tree
dt_params = {'max_depth': [3, 5, 7, 10, None], 'min_samples_split': [2, 5, 10]}
dt_grid = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_params, cv=5, scoring='accuracy')
dt_grid.fit(X_train, y_train)
best_dt = dt_grid.best_estimator_
print(f"Best Decision Tree Params: {dt_grid.best_params_}")


## 7. Evaluation

In [ ]:

def evaluate_model(model, X, y, name):
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1]
    
    acc = accuracy_score(y, y_pred)
    fpr, tpr, _ = roc_curve(y, y_prob)
    roc_auc = auc(fpr, tpr)
    
    print(f"--- {name} ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"AUC: {roc_auc:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(y, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.title(f"Confusion Matrix - {name}")
    plt.show()
    
    return fpr, tpr, roc_auc

# Evaluate Logistic Regression
fpr_lr, tpr_lr, auc_lr = evaluate_model(best_lr, X_test_scaled, y_test, "Logistic Regression")

# Evaluate Decision Tree
fpr_dt, tpr_dt, auc_dt = evaluate_model(best_dt, X_test, y_test, "Decision Tree")

# ROC Curve Comparison
plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, label=f"Logistic Regression (AUC={auc_lr:.2f})")
plt.plot(fpr_dt, tpr_dt, label=f"Decision Tree (AUC={auc_dt:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.show()


## 8. Feature Importance

In [ ]:

# Feature Importance for Logistic Regression
coefs = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': best_lr.coef_[0]
})
coefs['Abs_Coefficient'] = coefs['Coefficient'].abs()
coefs = coefs.sort_values(by='Abs_Coefficient', ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficient', y='Feature', data=coefs, palette='viridis')
plt.title("Top 10 Features (Logistic Regression)")
plt.show()



## 9. Conclusion
- We successfully built a pipeline to predict heart disease risk.
- **Logistic Regression** achieved an AUC of approximately **0.90**, making it a strong candidate for this task.
- Key risk factors identified include **Chest Pain Type (cp)**, **Thalach (Max Heart Rate)**, and **Oldpeak**.
